In [ ]:
!pip install transformers
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 2.1MB 7.9MB/s 
     |████████████████████████████████| 3.3MB 38.5MB/s 
     |████████████████████████████████| 901kB 39.9MB/s 
Mounted at /content/drive


In [ ]:
import sys
sys.path.insert(0, "//")

In [ ]:
import numpy as np
import time
import torch
import os
import torch.nn as nn
import time
import gc
import copy
import re
import pandas as pd
import torch.nn.functional as F
import torch.optim as optim
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = "cuda" if torch.cuda.is_available() else "cpu"
print(sys.path)
from content.data_extractor import DataExtractor
from content.data_extractor import TorchDataset

['//', '', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


In [ ]:
torch.cuda.is_available()

True

# Dataset

In [ ]:
dataset = TorchDataset(
    file_name="./data/diverse.triplets.train.tsv",
    queries_path="./data/diverse.queries.all.tsv",
    passages_path="./data/diverse.passages.all.tsv",
)
type(dataset)

content.data_extractor.TorchDataset

In [30]:
dataset.__getitem__(9000)

('heir apparent main character',
 "Penrod is Princess Janine's guard. He is protective, loyal, strong, a good listener, and respectful. If I had to choose one of the three main characters in Heir Apparent to be hang out with, I would choose Giannine. She is brave and seems to always have her friends' and family's back.",
 'A short story (usually) has one main character. It is not unusual for there to be only one character. There can be other SECONDARY characters, but all in a short story has a very limited number of characters. You are correct: central character and main character are the same thing.')

In [ ]:
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    DistilBertForMaskedLM,
)
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [ ]:
batch_size = 8
train_dataset = TorchDataset(
    file_name="./data/diverse.triplets.train.tsv",
    queries_path="./data/diverse.queries.all.tsv",
    passages_path="./data/diverse.passages.all.tsv",
)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

dev_dataset = TorchDataset(
    file_name="./data/diverse.triplets.dev.tsv",
    queries_path="./data/diverse.queries.all.tsv",
    passages_path="./data/diverse.passages.all.tsv",
)
dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TorchDataset(
    file_name="./data/diverse.triplets.test.tsv",
    queries_path="./data/diverse.queries.all.tsv",
    passages_path="./data/diverse.passages.all.tsv",
)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Two approaches:
- A binary classier 
- A model generate vector representation for queries and documents



In [ ]:
model_path = '/content/drive/MyDrive/reranking/'
NUM_EPOCHS = 1
LEARNING_RATE=0.00003

## Classification-based approach
- Data: query, positive_document, negative_document
- Training with entropy loss as a classification problem; replicate the CMU paper
- Inference: if a document is relevant or not relevant

### Train/Evaluation

In [ ]:
NUM_EPOCHS = 1
LEARNING_RATE = 0.00003

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2)
model.to(device)
model.train()

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

def predict(inputs, model, tokenizer, labels):
    encodings = tokenizer(
        inputs,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512,
    )
    ids, masks = encodings["input_ids"], encodings["attention_mask"]
    outputs = model(ids.to(device), masks.to(device), labels=labels.to(device))
    return outputs


dataloader = train_dataloader
N = len(dataloader)
lowest_loss = float("inf")
start = time.time()
learning_curve_y = []
learning_curve_x = []


for epoch in range(NUM_EPOCHS):
    epoch_loss = 0
    for i, (queries, pos_docs, neg_docs) in enumerate(dataloader):

        optimizer.zero_grad()  # set gradient to zero

        queries = list(queries) * 2  # 2*B
        docs = list(pos_docs) + list(neg_docs)

        labels = torch.cat([torch.ones(len(pos_docs)), 
                            torch.zeros(len(neg_docs))]).long().to(device)  # 2*batch,

        outputs = predict(
            inputs=list(zip(queries, docs)),
            model=model,
            tokenizer=tokenizer,
            labels=labels,
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        epoch_loss += float(loss)

        if i % 10 == 0:
            elapsed_time = time.time() - start
            remaining_time = elapsed_time * (1 / (i + 1) * N - 1)
            print(
                f"{i}: remaining time: {remaining_time:.1f} | est. epoch loss: {epoch_loss / (i + 1):.4f}"
            )

        if i % 10 == 0:
            with torch.no_grad():
                correct = total = 0
                val_start = time.time()
                for dq, dp, dn in dev_dataloader:
                    queries = list(dq) * 2  # 2*B
                    docs = list(dp) + list(dn)
                    labels = torch.cat([torch.ones(len(dp)), 
                             torch.zeros(len(dn))]).long().to(device)
                    outputs = predict(
                        inputs=list(zip(queries, docs)),
                        model=model,
                        tokenizer=tokenizer,
                        labels=labels
                    )

                    predicted_classes = outputs.logits.argmax(dim=-1)
                    correct += (labels == predicted_classes).sum()

                    total += len(labels)
                    if time.time() - val_start > 15:
                        break
                print(f"{i}: est. validation accuracy: {correct/total:.4f}")
                learning_curve_y.append(correct/total)
                learning_curve_x.append(i * batch_size)  # epoch normally 

        if (epoch_loss / (i + 1)) < lowest_loss:
            torch.save(model.state_dict(), "model1_best.pt")
            lowest_loss = epoch_loss / (i + 1)

    print(f"loss for epoch {epoch} is {epoch_loss}")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

0: remaining time: 2041.4 | est. epoch loss: 0.6830
0: est. validation accuracy: 0.5130
10: remaining time: 13611.2 | est. epoch loss: 0.6953
10: est. validation accuracy: 0.5748
20: remaining time: 14033.5 | est. epoch loss: 0.6946
20: est. validation accuracy: 0.5957
30: remaining time: 14163.8 | est. epoch loss: 0.6888
30: est. validation accuracy: 0.6323
40: remaining time: 14237.9 | est. epoch loss: 0.6897
40: est. validation accuracy: 0.6575
50: remaining time: 14270.5 | est. epoch loss: 0.6873
50: est. validation accuracy: 0.6750
60: remaining time: 14256.6 | est. epoch loss: 0.6821
60: est. validation accuracy: 0.6827
70: remaining time: 15634.0 | est. epoch loss: 0.6750
70: est. validation accuracy: 0.7084
80: remaining time: 16299.3 | est. epoch loss: 0.6618
80: est. validation accuracy: 0.7273
90: remaining time: 17515.3 | est. epoch loss: 0.6436
90: est. validation accuracy: 0.7253
100: remaining time: 17921.5 | est. epoch loss: 0.6298
100: est. validation accuracy: 0.7521


### Additional Experiments:
- Experiment1: No training, directly use Bert pretrained model
- Expeirment2: Free transformer layers, only train MLP


In [ ]:
# Experiment1: 
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2)

## Use this model to test on dev set  

In [ ]:
# Experiment2: 
# there are 4 layers in this model
import copy
model.load_state_dict(torch.load(model_path+"model1_best.pt"))
model_frozen = copy.deepcopy(model)
for param in model_frozen.distilbert.parameters():
  param.requires_grad = False

## Use the model_frozen to repeat the Train/Evaluate step

## Distance-based approach
- Data: query, positive_document, negative_document
- Training with triplet loss to compute doc-query distance
- Inference: score for each document to the given query 


### Training/Evaluation

In [ ]:
# DistilBertForSequenceClassification
# Experiment3: use DistilBertForMaskedLM instead of DistilBertForSequenceClassification in the model variable below
# DistilBertForMaskedLM
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
model.to(device)
model.train()

optimizer = torch.optim.Adam(model.distilbert.parameters(), lr=LEARNING_RATE)
# Experiment1: simply change margin to 0.1 here
triplet_loss = nn.TripletMarginLoss(margin=1.0)

# Experiment2: using the cosine distance function, use this triple_loss instead of the above one
# triplet_loss = nn.TripletMarginWithDistanceLoss(
#     distance_function=lambda x, y: 1 - F.cosine_similarity(x, y, dim=-1),
#     margin=1.0,
# )

def predict(inputs, model, tokenizer):
    encodings = tokenizer(
        inputs,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512,
    )
    ids, masks = encodings["input_ids"], encodings["attention_mask"]
    outputs = model.distilbert(ids.to(device), masks.to(device))
    outputs_hidden = outputs.last_hidden_state[:,0] #mean(dim=1)
    # Experiment3： using the average of all nodes in the last transfomer layer, use the below outputs_hidden
    # outputs_hidden = outputs.last_hidden_state.mean(dim=1)

    return outputs_hidden.view(3, len(queries), -1)


dataloader = train_dataloader
N = len(dataloader)
lowest_loss = float("inf")
start = time.time()
learning_curve_y = []
learning_curve_x = []


for epoch in range(NUM_EPOCHS):
    epoch_loss = 0
    for i, (queries, pos_docs, neg_docs) in enumerate(dataloader):

        optimizer.zero_grad()  # set gradient to zero
        anchors, positives, negatives = predict(
            inputs=list(queries + pos_docs + neg_docs),
            model=model,
            tokenizer=tokenizer,
        )

        loss = triplet_loss(anchors, positives, negatives)
        loss.backward()
        optimizer.step()

        epoch_loss += float(loss)

        if i % 10 == 0:
            elapsed_time = time.time() - start
            remaining_time = elapsed_time * (1 / (i + 1) * N - 1)
            print(
                f"{i}: remaining time: {remaining_time:.1f} | est. epoch loss: {epoch_loss / (i + 1):.4f}"
            )

        if i % 10 == 0:
            with torch.no_grad():
                correct = total = 0
                val_start = time.time()
                for dq, dp, dn in dev_dataloader:
                    anchors, positives, negatives = predict(
                        inputs=list(dq + dp + dn),
                        model=model,
                        tokenizer=tokenizer,
                    )

                    # using l2 norm distance
                    pos_dist = (anchors - positives).norm(dim=-1)  # B distances
                    neg_dist = (anchors - negatives).norm(dim=-1)  # B distances
                    
                    # Experiment2: using cosine distance function: 
                    # simply comment out the 2 lines above, and use the following 2 lines instead
                    # pos_dist = 1 - F.cosine_similarity(anchors, positives, dim=-1)
                    # neg_dist = 1 - F.cosine_similarity(anchors, negatives, dim=-1)

                    correct += float((pos_dist < neg_dist).sum())
                    total += len(dq)
                    if time.time() - val_start > 15:
                        break
                print(f"{i}: est. validation accuracy: {correct/total:.4f}")
                learning_curve_y.append(correct/total)
                learning_curve_x.append(i * batch_size)  # epoch normally 

        if (epoch_loss / (i + 1)) < lowest_loss:
            torch.save(model.state_dict(), "model2_best.pt")
            lowest_loss = epoch_loss / (i + 1)

    print(f"loss for epoch {epoch} is {epoch_loss}")

### Additional Experiments:
- Experiment1: Different TripletMarginLoss margins: 1 change: change the `margin` inside `triplet_loss` from 1.0 to 0.1
- Experiment2: Different TripletMarginLoss distance functions: cosine instead of l2 norm: 2 changes: 
  - 1) change `triplet_loss`; 
  - 2) change `pos_dist` and `neg_dist`
- Experiment3: average of final transformer layer instead of just first: 2 changes:
  - change `model` to `DistilBertForMaskedLM`
  - change `outputs_hidden`
- Experiment4: only use pretrained model with no further training (cell below)

In [ ]:
# Experiment4: BERT (no training)
model2_exp4 = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to(device)
# From here, do evaluation on the dev set


### Testing using distance-based model

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
tokenizer_options = {
    "return_tensors": "pt",
    "truncation": True,
    "padding": True,
    "max_length": 512,
}

model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")
model.load_state_dict(torch.load(model_path + "demo_model.pt", map_location=device))
model.to(device)
model.eval()

DistilBertForMaskedLM(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): L

In [29]:
queries = test_dataset.queries[0:1] 
documents = test_dataset.positive_doc[0:1] + test_dataset.negative_doc[:4] + ["UBC MDS is awesome?"]

with torch.no_grad():
    inputs = queries + documents

    # Tokenize queries and documents 
    encodings = tokenizer(inputs, **tokenizer_options).to(device)
    ids, masks = encodings["input_ids"], encodings["attention_mask"]

    # Inference pass through model
    outputs = model.distilbert(ids, masks)
    outputs_hidden = outputs.last_hidden_state.mean(dim=1) #[:, 0]
    vec_queries = outputs_hidden[:len(queries)].unsqueeze(1)
    vec_documents = outputs_hidden[len(queries):].unsqueeze(0)

    # Compute pairwise distances between queries and documents
    all_distances, all_indices = (vec_queries - vec_documents).norm(dim=-1).sort(dim=-1)
    
    # Print nicely the results
    for qdistances, dindices, query in zip(all_distances, all_indices, queries):
        # display(Markdown(f"### {query}"))
        print(query)
        for dist, dindex in zip(qdistances, dindices):
            # display(Markdown(f"**{dist:.2f}**: {documents[dindex]}"))
            print(dist.item(), documents[dindex])


      


definition of rent in economics
9.83952808380127 Rent, in economics, the income derived from the ownership of land and other free gifts of nature. The neoclassical economist Alfred Marshall, and others after him, chose this definition for technical reasons, even though it is somewhat more restrictive than the meaning given the term in popular usage.
11.80168628692627 First, the Rent Court will only award a money judgment for the rent due through the date of trial; a landlord cannot obtain future rent or liquidated damages for rent coming due past the trial date. If the landlord wants to collect future rent, it will have to file a separate collection action.
12.555009841918945 The acres of land available for rent (supply) and the number of tenants wanting to rent for cash (demand), as well as production costs and market prices, can affect the amount of rent negotiated. Cash rent paid for Missouri crop and pasture land in 2015.
13.303638458251953 Economics is the social science that anal